GreenMart Retail--Limpieza de datos

In [ ]:
#Librerías necesarias
import pandas as pd
import numpy as np
from google.colab import files
from sklearn.preprocessing import StandardScaler, LabelEncoder

#Archivos
uploaded = files.upload()

#Dataset
df = pd.read_csv("greenmart_customers_products.csv")

#Exploración del dataset

df.head() #primeras filas
df.info() #tipos de datos y valores faltantes
df.describe() #estadísticas descriptivas
list(df.columns) #listado de columnas

Saving greenmart_customers_products (2).csv to greenmart_customers_products (2) (1).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10400 entries, 0 to 10399
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        10400 non-null  int64  
 1   customer_name      9678 non-null   object 
 2   age                9678 non-null   float64
 3   city               9678 non-null   object 
 4   product_id         10400 non-null  int64  
 5   product_name       10400 non-null  object 
 6   category           10400 non-null  object 
 7   purchase_date      9678 non-null   object 
 8   purchase_quantity  9692 non-null   float64
 9   price_per_unit     10400 non-null  float64
 10  total_spent        9692 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 893.9+ KB


['customer_id',
 'customer_name',
 'age',
 'city',
 'product_id',
 'product_name',
 'category',
 'purchase_date',
 'purchase_quantity',
 'price_per_unit',
 'total_spent']

In [ ]:
#Identificación y posición de valores faltantes
df.isnull().sum()

,0
customer_id,0
customer_name,722
age,722
city,722
product_id,0
product_name,0
category,0
purchase_date,722
purchase_quantity,708
price_per_unit,0


Las siguientes columnas no nos aportan información utilizable y transferible, ya que están prácticamente vacías:
- customer_name
- age
- city
- purchase_date

Cuando una columna tiene más del 50-60% de valores faltantes, se considera no fiable.
Por ende, la mejor opción es la eliminación de las mismas.

In [ ]:
#Eliminar filas con demasiados nulls
cols_to_drop = ["customer_name", "age", "city"]
df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors="ignore")



,customer_id,product_id,product_name,category,purchase_date,purchase_quantity,price_per_unit,total_spent
0,1,141,Granola,Vegetable,2024-10-21,1.0,13.33,13.33
1,2,172,Organic Apples,Vegetable,2024-08-15,2.0,1.04,2.08
2,3,186,Granola,Dairy,2024-07-16,1.0,15.33,15.33
3,4,110,Granola,Snack,2024-09-18,1.0,10.55,10.55
4,5,147,Fresh Milk,Snack,2024-10-09,1.0,12.25,12.25
...,...,...,...,...,...,...,...,...
10395,9745,191,Kale,Organic,2024-05-16,1.0,15.82,15.82
10396,9251,162,Almond Butter,Organic,2024-04-11,2.0,9.46,18.92
10397,8525,186,Tofu,Vegetable,2024-04-01,2.0,6.51,13.02
10398,4929,125,Organic Apples,Organic,2024-09-27,2.0,2.03,4.06


In [ ]:
#Cuántos duplicados hay
df.duplicated().sum()


#Eliminar duplicados
df = df.drop_duplicates()

In [ ]:
#Normalizar fechas(Purchase_date)
if "purchase_date" in df.columns:
  df["purchase_date"] = pd.to_datetime(df["purchase_date"], errors="coerce")
  df["year"] = df["purchase_date"].dt.year
  df["month"] = df["purchase_date"].dt.month
  df["day"] = df["purchase_date"].dt.day
  df["weekday"] = df["purchase_date"].dt.weekday # 0 = Lunes

#Eliminamos la columna fecha original
df.drop(columns=["purchase_date"])

,customer_id,customer_name,age,city,product_id,product_name,category,purchase_quantity,price_per_unit,total_spent,year,month,day,weekday
0,1,Ronald Miller,63.0,Wilsonstad,141,Granola,Vegetable,1.0,13.33,13.33,2024.0,10.0,21.0,0.0
1,2,Eric White,26.0,North Brianside,172,Organic Apples,Vegetable,2.0,1.04,2.08,2024.0,8.0,15.0,3.0
2,3,Joseph Wagner,59.0,Lake Derek,186,Granola,Dairy,1.0,15.33,15.33,2024.0,7.0,16.0,1.0
3,4,Elizabeth House,43.0,Cindyborough,110,Granola,Snack,1.0,10.55,10.55,2024.0,9.0,18.0,2.0
4,5,Shawn Warner,56.0,North Hannah,147,Fresh Milk,Snack,1.0,12.25,12.25,2024.0,10.0,9.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10369,598,Kelly Hill,64.0,Daymouth,111,Granola,Snack,3.0,9.34,28.02,2024.0,8.0,20.0,1.0
10379,47,Heather Keller,62.0,Wesq Stephen,125,ALMOND BUTTER,Dairy,1.0,12.27,12.27,2024.0,5.0,5.0,6.0
10386,6341,Kevin Reyes,76.0,Randolphfurt,114,Tofu,Dairy,1.0,7.72,7.72,2024.0,8.0,16.0,4.0
10388,2388,Mr. Anthony Crawford PhD,38.0,Port Emily,115,Fresh Milk,Vegetable,3.0,19.93,59.79,2024.0,10.0,31.0,3.0


Normalizar valores numéricos del dataset:
- purchase_quantity
- price_per_unit
- total_spent

Vamos a estandarizarlas con Standard Scaler(media=0 y desviación=1)


In [ ]:
#Normalizar valores numéricos
num_cols = [col for col in["purchase_quantity", "price_per_unit", "total_spent"]]

scaler = StandardScaler()

df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
#Convertimos cada categoría en un número único
cols_to_encode = [col for col in["product_name", "category"] if col in df.columns]
le = LabelEncoder()
for col in cols_to_encode:
    df[col] = le.fit_transform(df[col])


In [ ]:
#Guardar el dataset limpio
df.to_csv("Greenmart_dataset_limpio.csv", index=False)

#descargar el archivo limpio
files.download("Greenmart_dataset_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>